In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import timedelta
import xgboost as xgb

### Load Datasets

In [ ]:
calendar_df = pd.read_csv('data/calendar.csv')
inventory_df = pd.read_csv('data/inventory.csv')
sales_test_df = pd.read_csv('data/sales_test.csv')
sales_train_df = pd.read_csv('data/sales_train.csv')
solution_df = pd.read_csv('data/solution.csv')
test_weights_df = pd.read_csv('data/test_weights.csv')

In [ ]:
sales_train_df = sales_train_df.drop(columns=['availability'])
sales_train_df.dropna(subset=['sales'], inplace=True)
sales_test_df['sales'] = 0
df = pd.concat([sales_train_df, sales_test_df], ignore_index=True).sort_values('date')
df = df.merge(calendar_df, on=['date', 'warehouse'], how='left')
df = df.merge(inventory_df, on=['unique_id', 'warehouse'], how='left')
df['date'] = pd.to_datetime(df['date'])

### Feature Engineering

In [ ]:
periods = [1,2,3,4,5,6,7,14,21,28,60,90,120,180,270,360]

In [ ]:
def add_date_features(df):
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['year'] = df['date'].dt.year
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['weekday'] = df['date'].dt.weekday
    df['day_of_year'] = df['date'].dt.dayofyear
    df['year_sin'] = np.sin((df['year'] - df['year'].min()) / (df['year'].max() - df['year'].min()) * 2 * np.pi)
    df['month_sin'] = np.sin(df['month'] / 12 * 2 * np.pi)
    df['year_cos'] = np.cos((df['year'] - df['year'].min()) / (df['year'].max() - df['year'].min()) * 2 * np.pi)
    df['month_cos'] = np.cos(df['month'] / 12 * 2 * np.pi)
    return df

def add_product_cat(df):
    df['category'] = df['name'].str.split('_', expand=True)[0]
    return df

def add_lagged_product_sales(df):
    for period in periods:
        df[f'sales_{period}'] = df.groupby(['warehouse', 'name'])['sales'].shift(periods=period)
    return df 

def add_order_difference(df):
    for period in periods:
        df[f'orders_{period}'] = df.groupby(['warehouse', 'name'])['total_orders'].shift(periods=period)
        df[f'orders_diff_{period}'] = df['total_orders'] - df[f'orders_{period}']
    return df

def add_country_and_subcountry(df):
    warehouse_to_country = {
        'Budapest_1': 'Hungary',
        'Prague_1': 'Czech Republic',
        'Prague_2': 'Czech Republic',
        'Prague_3': 'Czech Republic',
        'Brno_1': 'Czech Republic',
        'Munich_1': 'Germany',
        'Frankfurt_1': 'Germany'
    }

    warehouse_to_subcountry = {
        'Budapest_1': 'Hungary',
        'Prague_1': 'Czech Republic 1',
        'Prague_2': 'Czech Republic 2',
        'Prague_3': 'Czech Republic 2',
        'Brno_1': 'Czech Republic 1',
        'Munich_1': 'Germany 1',
        'Frankfurt_1': 'Germany 2'
    }

    # Add a country column using the mapping
    df['country'] = df['warehouse'].map(warehouse_to_country)
    
    # Add a country column using the mapping
    df['subcountry'] = df['warehouse'].map(warehouse_to_subcountry)

    return df
    
    
def add_discount_features(df):
    df['best_discount'] = df[['type_0_discount', 'type_1_discount','type_2_discount', 'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount']].max(axis=1)

    df['type_0_discount_subcountry'] = df['subcountry']
    df['type_1_discount_subcountry'] = df['subcountry']
    df['type_2_discount_subcountry'] = df['subcountry']
    df['type_3_discount_subcountry'] = df['subcountry']
    df['type_4_discount_subcountry'] = df['subcountry']
    df['type_5_discount_subcountry'] = df['subcountry']
    df['type_6_discount_subcountry'] = df['subcountry']
    
    return df

def add_processed_holiday(df):
    holiday_mappings = {
        'Hungary': [
            'Good Friday',
            'Easter Monday',
            'Whit Monday',
            'Whit Sunday'
            'Christmas Eve',
            '1st Christmas Day',
            '2nd Christmas Day',
            'New Years Day',
            'Assumption of the Virgin Mary'
        ],
        'Czech Republic': [
            'Good Friday',
            'Easter Monday',
            'Whit Sunday',
            'Whit Monday',
            'Christmas Eve',
            '1st Christmas Day',
            '2nd Christmas Day',
            'New Years Day',
            'Epiphany',
            'Cyrila a Metodej'
        ],
        'Germany': [
            'Good Friday',
            'Easter Monday',
            'Corpus Christi',
            'Whit Sunday',
            'Whit Monday',
            'Christmas Eve',
            '1st Christmas Day',
            '2nd Christmas Day',
            'New Years Day',
            'Ascension day',
            'Assumption of the Virgin Mary',
            'Reformation Day'
        ]
    }

    df['holiday_name'] = df['holiday_name'].fillna('No Holiday')

    df['holiday'] = df.apply(
        lambda row: 1 if row['country'] in holiday_mappings and row['holiday_name'] in holiday_mappings[row['country']] else 0,
        axis=1
    )

    # Get rows where holiday == 1
    holiday_rows = df[df['holiday'] == 1].drop_duplicates(subset='date', ignore_index=True)
    
    # Set holidays in the range 3 days before and 1 day after, within the same subcountry
    for _, row in holiday_rows.iterrows():
        start_date = row['date'] - timedelta(days=3)
        end_date = row['date'] + timedelta(days=1)
        country = row['country']  # Get the country for the holiday row
        
        # Update rows in the date range and the same country
        df.loc[
            (df['date'] >= start_date) & 
            (df['date'] <= end_date) & 
            (df['country'] == country),
            'holiday'
        ] = 1

    return df

def add_target_encoding(df):

    df['target_l1_cat_trend'] = (
        df
        .groupby(['subcountry', 'L1_category_name_en'])['total_orders']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
    )

    df['target_l1_cat_daily'] = (
        df.groupby(['date', 'subcountry', 'L1_category_name_en'])['total_orders']
        .transform('mean')
    )

    df['target_l2_cat_trend'] = (
        df
        .groupby(['subcountry', 'L2_category_name_en'])['total_orders']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
    )

    df['target_l2_cat_daily'] = (
        df.groupby(['date', 'subcountry', 'L2_category_name_en'])['total_orders']
        .transform('mean')
    )

    df['target_l3_cat_trend'] = (
        df
        .groupby(['subcountry', 'L3_category_name_en'])['total_orders']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
    )

    df['target_l3_cat_daily'] = (
        df.groupby(['date', 'subcountry', 'L3_category_name_en'])['total_orders']
        .transform('mean')
    )

    df['target_l4_cat_trend'] = (
        df
        .groupby(['subcountry', 'L4_category_name_en'])['total_orders']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
    )

    df['target_l4_cat_daily'] = (
        df.groupby(['date', 'subcountry', 'L4_category_name_en'])['total_orders']
        .transform('mean')
    )

    df['target_category_trend'] = (
        df
        .groupby(['subcountry', 'category'])['total_orders']
        .expanding()
        .mean()
        .reset_index(level=[0, 1], drop=True)
    )

    df['target_category_daily'] = (
        df.groupby(['date', 'subcountry', 'category'])['total_orders']
        .transform('mean')
    )

    return df

In [ ]:
df = add_date_features(df)
df = add_product_cat(df)
df = add_lagged_product_sales(df)
df = add_order_difference(df)
df = add_country_and_subcountry(df)
df = add_processed_holiday(df)
df = add_discount_features(df)
df = add_target_encoding(df)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
# Sanity check
df[(df.name=='Croissant_36')&(df.warehouse=="Brno_1")][['warehouse','name','sales','sales_14', 'type_0_discount_subcountry', 'country', 'subcountry', 'orders_diff_14', 'orders_14', 'target_category_trend']].head(20)

In [ ]:
## Ensuring correct datatypes
for col in df.select_dtypes("object").columns:
    df[col] = df[col].astype('category')

### Model Training & Evaluation

In [ ]:
print(sales_train_df['date'].min())
print(sales_train_df['date'].max())
print(sales_test_df['date'].min())
print(sales_test_df['date'].max())

In [ ]:
train_start_date  = '2020-08-01'
train_end_date  = '2024-06-02'

train = df[(df.index >= train_start_date) & (df.index <= train_end_date)]
test  = df[(df.index >  train_end_date)]

X_train = train.drop(['sales'], axis=1)
y_train = train['sales']

X_test = test.drop(['sales'], axis=1)

In [ ]:
## Parameter from here: https://www.kaggle.com/code/meryentr/rohlik-sales-lightgbm-lb-20-75

lightgbm_params={ 
    'learning_rate': 0.021796506746095975,
    'num_leaves': 93,
    'max_depth': 10,
    'min_child_samples': 25,
    'subsample': 0.7057135664023435,
    'colsample_bytree': 0.8528497905459008,
    'reg_alpha': 0.036786449788597686,
    'reg_lambda': 0.3151110021900479,
    'num_boost_round': 9800,
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
}

xgb_params = {
    'learning_rate': 0.021796506746095975,  
    'max_leaves': 93,
    'n_estimators': 9800,  
    'max_depth': 10,  
    'min_child_weight': 25, 
    'subsample': 0.7057135664023435,  
    'colsample_bytree': 0.8528497905459008,  
    'reg_alpha': 0.036786449788597686, 
    'reg_lambda': 0.3151110021900479,  
    'objective': 'reg:squarederror',  
    'eval_metric': 'mae', 
    'tree_method': 'hist', 
}


In [ ]:
# Main LightGBM Model
main_lightgbm_model = lgb.LGBMRegressor(**lightgbm_params)
main_lightgbm_model.fit(X_train, y_train)
main_lightgbm_model.booster_.save_model("main_lightgbm_model.json")

In [ ]:
feature_importance_df = pd.DataFrame({'name':main_lightgbm_model.feature_name_})
feature_importance_df['importance'] = main_lightgbm_model.feature_importances_
feature_importance_df['group'] = feature_importance_df['name'].apply(
    lambda x: 'lagged sales features' if 'sales_' in x else 'other features')
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 20))
ax = sns.barplot(data=feature_importance_df, x='importance', y='name',  hue='group', dodge=False)
ax.set_title(f"Feature importances")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

In [ ]:
recursive_country_features = feature_importance_df.iloc[:20]['name'].tolist()
print(recursive_country_features)

In [ ]:
def recursive_forecast_lgbm(country, X_test, X_train, unique_dates, train, recursive_country_features):
    results = []
    X_test_copy = X_test.copy()

    curr_X_train = X_train[X_train.country == country][recursive_country_features]
    curr_y_train = train[train.country == country]['sales']
    curr_X_test = X_test_copy[X_test_copy.country == country][recursive_country_features]

    for next_date in unique_dates:
        model = lgb.LGBMRegressor(**lightgbm_params)
        model.fit(curr_X_train, curr_y_train)

        # Predict for current date
        curr_X = curr_X_test.loc[next_date]
        pred_sales = model.predict(curr_X)

        # Create new data to train the model with
        new_X = curr_X.copy()
        pred_sales[pred_sales < 0] = 0
        results.append(pd.Series(pred_sales, index=curr_X.index))
        new_y = pd.Series(pred_sales, index=curr_X.index, name='sales')

        # Add new data back to original data
        curr_X_train = pd.concat([curr_X_train, new_X], ignore_index=False)
        curr_y_train = pd.concat([curr_y_train, new_y], ignore_index=False)

    forecast_df = pd.concat(results, axis=0).to_frame(name=f'{country}_pred_sales')
    curr_X_test = X_test_copy[X_test_copy.country == country] # Redefine current X test to merge
    curr_X_test['id'] = curr_X_test['unique_id'].astype(str) + "_" + curr_X_test.index.astype(str)

    merged_df = pd.concat([curr_X_test , forecast_df], axis=1)[['id', f'{country}_pred_sales']]

    model.booster_.save_model(f"lightgbm_model_{country}_rec.json")
    merged_df.to_csv(f'{country}_rec_predictions.csv')

    return model, merged_df

In [ ]:
def check_model_performance(y_pred):
    print(f"Mean of y_pred: {np.mean(y_pred):.4f}")
    print(f"Variance of y_pred: {np.var(y_pred):.4f}")
    print("="*50)

In [ ]:
countries = X_train['country'].unique()
unique_dates = X_test.index.unique()
country_rec_preds = {}
country_rec_model = {}

for country in countries:
    curr_model, curr_y_pred = recursive_forecast_lgbm(country, X_test, X_train, unique_dates, train, recursive_country_features)
    country_rec_preds[country] = curr_y_pred
    country_rec_model[country] = curr_model
    check_model_performance(curr_y_pred[f'{country}_pred_sales'].tolist())

In [ ]:
xgboost_features = feature_importance_df.iloc[:30]['name'].tolist()
print(xgboost_features)

In [ ]:
# Main XGB Model
main_xgb_model = xgb.XGBRegressor(**xgb_params, enable_categorical=True)
main_xgb_model.fit(X_train[xgboost_features], y_train)
main_xgb_model.save_model("main_xgb_model.json")

In [ ]:
# Train one model per country
country_models = {}

countries = X_train['country'].unique()
for country in countries:
    curr_X_train = X_train[X_train.country == country]
    curr_y_train = train[train.country == country]['sales']
    model = lgb.LGBMRegressor(**lightgbm_params)
    model.fit(curr_X_train, curr_y_train)
    country_models[country] = model
    model.booster_.save_model(f"lightgbm_model_{country}.json")

### Ensemble Solution Analysis

In [ ]:
X_test_copy = X_test.copy()
X_test_copy['id'] = X_test_copy['unique_id'].astype(str) + "_" + X_test_copy.index.astype(str)

In [ ]:
main_lightgbm_y_pred = main_lightgbm_model.predict(X_test)
main_lightgbm_y_pred[main_lightgbm_y_pred<0] = 0

main_lightgbm_y_pred_df = pd.DataFrame(main_lightgbm_y_pred, columns=['Main_LGBM_pred'], index=X_test_copy.index)
main_lightgbm_df = pd.concat([X_test_copy['id'], main_lightgbm_y_pred_df], axis=1)

In [ ]:
main_xgb_y_pred = main_xgb_model.predict(X_test[xgboost_features])
main_xgb_y_pred[main_xgb_y_pred<0] = 0

main_xgb_y_pred_df = pd.DataFrame(main_xgb_y_pred, columns=['Main_XGB_pred'], index=X_test_copy.index)
main_xgb_df = pd.concat([X_test_copy['id'], main_xgb_y_pred_df], axis=1)

In [ ]:
check_model_performance(main_lightgbm_y_pred)
check_model_performance(main_xgb_y_pred)

In [ ]:
country_preds = {}
for country, country_model in country_models.items():
    curr_X_test = X_test[X_test.country == country]
    curr_lightgbm_y_pred = country_model.predict(curr_X_test)
    curr_lightgbm_y_pred[curr_lightgbm_y_pred<0] = 0  
    print(country)
    check_model_performance(curr_lightgbm_y_pred)  

    curr_X_test['id'] = curr_X_test['unique_id'].astype(str) + "_" + curr_X_test.index.astype(str)
    curr_lightgbm_y_pred_df = pd.DataFrame(curr_lightgbm_y_pred, columns=[f'{country}_pred'], index=curr_X_test.index)
    curr_lightgbm_df = pd.concat([curr_X_test['id'], curr_lightgbm_y_pred_df], axis=1)
    country_preds[country] = curr_lightgbm_df

In [ ]:
country_preds = {}
countries = X_train['country'].unique()
for country in countries:
    country_model = lgb.Booster(model_file=f"lightgbm_model_{country}.json")
    curr_X_test = X_test[X_test.country == country]
    curr_lightgbm_y_pred = country_model.predict(curr_X_test)
    curr_lightgbm_y_pred[curr_lightgbm_y_pred<0] = 0  
    print(country)
    check_model_performance(curr_lightgbm_y_pred)  
    
    curr_X_test['id'] = curr_X_test['unique_id'].astype(str) + "_" + curr_X_test.index.astype(str)
    curr_lightgbm_y_pred_df = pd.DataFrame(curr_lightgbm_y_pred, columns=[f'{country}_pred'], index=curr_X_test.index)
    curr_lightgbm_df = pd.concat([curr_X_test['id'], curr_lightgbm_y_pred_df], axis=1)
    country_preds[country] = curr_lightgbm_df

### Scoring and Submitting

In [ ]:
test['id'] = test['unique_id'].astype(str) + "_" + test.index.astype(str)
test = test[['id']]
test = test.merge(main_lightgbm_df, on='id', how='left')
test = test.merge(main_xgb_df, on='id', how='left')

for country, country_pred in country_preds.items():
    test = test.merge(country_pred, on='id', how='left')

for country, country_rec_pred in country_rec_preds.items():
    test = test.merge(country_rec_pred, on='id', how='left')   

print(test.columns)

In [ ]:
test = test[['id', 'Main_LGBM_pred', 'Main_XGB_pred', 'Czech Republic_pred',
       'Hungary_pred', 'Germany_pred', 'Germany_pred_sales', 
       'Czech Republic_pred_sales', 'Hungary_pred_sales']] 
test['sales_hat'] = test.drop(columns=['id']).sum(axis=1, skipna=True) / 4
test[['id','sales_hat']].to_csv("submission.csv",index=False)